In [1]:
import pandas as pd
# Dependencies
# ----------------------------------
# Imports the method used for connecting to DBs
from sqlalchemy import create_engine

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float 

In [2]:
passing = 'NFL_STATISCS/Career_Stats_Passing.csv'
reciving = 'NFL_STATISCS/Career_Stats_Receiving.csv'
rushing = 'NFL_STATISCS/Career_Stats_Rushing.csv'



qb = 'NFL_STATISCS/Game_Logs_Quarterback.csv'
basic = 'NFL_STATISCS/Basic_Stats.csv'
runnungBack = 'NFL_STATISCS/Game_Logs_Runningback.csv'
receiver = 'NFL_STATISCS/Game_Logs_Wide_Receiver_and_Tight_End.csv'

In [3]:
#passing_df = pd.read_csv(file_passing)
#passing_df.loc[passing_df['Player Id']=='billdemory/2512778',:]

In [6]:
# Create all the DataFrames
qb_df = pd.read_csv(qb)             # Len = 40,247
runningBack_df = pd.read_csv(runnungBack) #Len = 67,661   nan, 'RB', 'FB'    #11,626 That Contain Position 
receiver_df = pd.read_csv(receiver)       #Len 112,632    nan, 'WR', 'TE',   # 24,786 That Contain Position 


#passing_df = pd.read_csv(passing) # Len = 8,525 First Year = 1924
#rushing_df = pd.read_csv(rushing) # Len = 17,507 First Year = 1924
#reciving_df = pd.read_csv(reciving) #Len = 18,128
#basic_df = pd.read_csv(basic)       #Len = 17,172



In [7]:
runningBack_df.columns

Index(['Player Id', 'Name', 'Position', 'Year', 'Season', 'Week', 'Game Date',
       'Home or Away', 'Opponent', 'Outcome', 'Score', 'Games Played',
       'Games Started', 'Rushing Attempts', 'Rushing Yards', 'Yards Per Carry',
       'Longest Rushing Run', 'Rushing TDs', 'Receptions', 'Receiving Yards',
       'Yards Per Reception', 'Longest Reception', 'Receiving TDs', 'Fumbles',
       'Fumbles Lost'],
      dtype='object')

In [8]:
receiver_df.columns

Index(['Player Id', 'Name', 'Position', 'Year', 'Season', 'Week', 'Game Date',
       'Home or Away', 'Opponent', 'Outcome', 'Score', 'Games Played',
       'Games Started', 'Receptions', 'Receiving Yards', 'Yards Per Reception',
       'Longest Reception', 'Receiving TDs', 'Rushing Attempts',
       'Rushing Yards', 'Yards Per Carry', 'Longest Rushing Run',
       'Rushing TDs', 'Fumbles', 'Fumbles Lost'],
      dtype='object')

In [9]:
#Cleaning the QB Data Frame

qb_df['Position'] = qb_df['Position'].fillna('QB')

qb_clean_df = qb_df.loc[(qb_df['Season']=='Regular Season') | (qb_df['Season']=='Postseason'),:].replace('--',0)
qb_clean_df

qb_clean_df = qb_clean_df.rename(columns = {'Player Id': 'Player_id',
                                           'Games Played':'Games_Played',
                                          'Games Started':'Games_Started',
                                            'Passes Completed' :'Passes_Completed',
                                            'Passes Attempted':'Passes_Attempted'})


columns = ['Player_id', 'Name', 'Position','Year', 'Season', 'Games_Played', 
           'Games_Started', 'Passes_Completed', 'Passes_Attempted']
qb_clean_df = qb_clean_df.reset_index(drop= True)
qb_df_sql = qb_clean_df.loc[:,columns]

qb_df_sql

,Player_id,Name,Position,Year,Season,Games_Played,Games_Started,Passes_Completed,Passes_Attempted
0,billdemory/2512778,"Demory, Bill",QB,1974,Regular Season,0,0,0,0
1,billdemory/2512778,"Demory, Bill",QB,1974,Regular Season,0,0,0,0
2,billdemory/2512778,"Demory, Bill",QB,1974,Regular Season,0,0,0,0
3,billdemory/2512778,"Demory, Bill",QB,1974,Regular Season,0,0,0,0
4,billdemory/2512778,"Demory, Bill",QB,1974,Regular Season,0,0,0,0
...,...,...,...,...,...,...,...,...,...
36181,steveramsey/2523725,"Ramsey, Steve",QB,1971,Regular Season,1,0,8,16
36182,steveramsey/2523725,"Ramsey, Steve",QB,1971,Regular Season,1,0,10,25
36183,steveramsey/2523725,"Ramsey, Steve",QB,1971,Regular Season,1,0,16,38
36184,steveramsey/2523725,"Ramsey, Steve",QB,1970,Regular Season,0,0,0,0


In [10]:
#DATA FRAME THAT contains unique players QB

qb_id = qb_df_sql['Player_id'].unique()
qb_name = qb_df_sql['Name'].unique()


#qb_id_name = pd.DataFrame(qb_id,qb_name, columns  = ['Player_id','Player_id'])
qb_id_name = pd.DataFrame({'Player_id': qb_id, 'Name' : qb_name})#, columns  = ['Player_id','Player_id'])
qb_id_name['Position'] = 'QB'
qb_id_name


,Player_id,Name,Position
0,billdemory/2512778,"Demory, Bill",QB
1,tomsavage/2543640,"Savage, Tom",QB
2,tommymaddox/2501842,"Maddox, Tommy",QB
3,jimbobtaylor/2527093,"Taylor, Jim Bob",QB
4,jaybarker/2499535,"Barker, Jay",QB
...,...,...,...
462,alpastrana/2522827,"Pastrana, Al",QB
463,aaronrodgers/2506363,"Rodgers, Aaron",QB
464,chrisweinke/2504686,"Weinke, Chris",QB
465,brockosweiler/2533436,"Osweiler, Brock",QB


In [14]:
# Cleaning RB Data Frame UNIQUE RUNNERS AND NANMES

df = runningBack_df.loc[(runningBack_df['Position']== 'RB') | (runningBack_df['Position']== 'FB'),:]
df
runners_df = df.loc[(df['Season']=='Regular Season') | (df['Season']=='Postseason'),:].reset_index(drop=True).replace('--',0)


rename_columns =  {'Player Id': 'Player_id',
                    'Games Played':'Games_Played',
                    'Games Started':'Games_Started',
                    'Rushing Yards' :'Passes_Completed',
                    'Rushing TDs':'Rushing_TDs',
                   'Receiving TDs':'Receiving_TDs',
                   'Receiving Yards':'Receiving_Yards'
                  }

runningBack_clean_df= runners_df.rename(columns = rename_columns)
columns = ['Player_id','Name','Position','Year','Season','Games_Played','Games_Started',\
           'Passes_Completed','Rushing_TDs','Receptions']
runners_df= runningBack_clean_df.loc[:,columns]
runners_df

,Player_id,Name,Position,Year,Season,Games_Played,Games_Started,Passes_Completed,Rushing_TDs,Receptions
0,kenyandrake/2555230,"Drake, Kenyan",RB,2016,Regular Season,1,0,0,0,0
1,kenyandrake/2555230,"Drake, Kenyan",RB,2016,Regular Season,1,0,12,1,2
2,kenyandrake/2555230,"Drake, Kenyan",RB,2016,Regular Season,1,1,37,0,2
3,kenyandrake/2555230,"Drake, Kenyan",RB,2016,Regular Season,1,0,15,0,0
4,kenyandrake/2555230,"Drake, Kenyan",RB,2016,Regular Season,1,0,4,0,2
...,...,...,...,...,...,...,...,...,...,...
8950,derekwatt/2555220,"Watt, Derek",FB,2016,Regular Season,1,0,2,0,0
8951,derekwatt/2555220,"Watt, Derek",FB,2016,Regular Season,1,0,0,0,0
8952,derekwatt/2555220,"Watt, Derek",FB,2016,Regular Season,1,0,0,0,0
8953,derekwatt/2555220,"Watt, Derek",FB,2016,Regular Season,1,0,0,0,1


In [18]:
#DATA FRAME THAT contains unique players RB and TE RUNNERS

runners_id = runners_df['Player_id'].unique()
runners_name = runners_df['Name'].unique()
runners_id_id_name = pd.DataFrame({'Player_id': runners_id, 'Name' : runners_id})
runners_id_id_name


,Player_id,Name
0,kenyandrake/2555230,kenyandrake/2555230
1,darylrichardson/2534789,darylrichardson/2534789
2,dwaynewashington/2556118,dwaynewashington/2556118
3,dujuanharris/2530690,dujuanharris/2530690
4,zachline/2539303,zachline/2539303
...,...,...
158,mikedavis/2553439,mikedavis/2553439
159,shanevereen/2495473,shanevereen/2495473
160,wendellsmallwood/2555461,wendellsmallwood/2555461
161,chrisivory/2507999,chrisivory/2507999


In [16]:
#receiver_df = pd.read_csv(receiver)       #Len 112,632    nan, 'WR', 'TE',   # 24,786 That Contain Position 


rec_df = receiver_df.loc[(receiver_df['Position']== 'WR') | (receiver_df['Position']== 'TE'),:]
rec_df
receiver_clean_df = rec_df.loc[(rec_df['Season']=='Regular Season') | (rec_df['Season']=='Postseason'),:]\
.reset_index(drop=True).replace('--',0)

rename_columns =  {'Player Id': 'Player_id',
                'Games Played':'Games_Played',
                'Games Started':'Games_Started',
                'Receiving Yards' : 'Receiving_Yards',
                'Receiving TDs':'Receiving_TDs',
                  'Rushing TDs':'Rushing_TDs',}


receiver_clean_df = receiver_clean_df.rename(columns=rename_columns)
#receiver_clean_df = receiver_clean_df.loc[:,receiver_clean_df['Games_Played','Name','Games_Played',
#                                                              'Games_Started','Receiving_Yards','Receiving_TDs']]

#columns = ['Player_id','Name','Position','Year','Season','Games_Played','Games_Started',\
#           'Passes_Completed','Rushing_TDs','Receptions']

columns = ['Player_id','Name','Position', 'Year', 'Season','Games_Played','Games_Started',\
           'Receptions','Receiving_Yards','Receiving_TDs']
receiver_clean_df = receiver_clean_df.loc[:,columns]
receiver_clean_df

,Player_id,Name,Position,Year,Season,Games_Played,Games_Started,Receptions,Receiving_Yards,Receiving_TDs
0,kevinnorwood/2543689,"Norwood, Kevin",WR,2015,Regular Season,0,0,0,0,0
1,kevinnorwood/2543689,"Norwood, Kevin",WR,2015,Regular Season,1,0,0,0,0
2,kevinnorwood/2543689,"Norwood, Kevin",WR,2015,Regular Season,0,0,0,0,0
3,kevinnorwood/2543689,"Norwood, Kevin",WR,2015,Regular Season,0,0,0,0,0
4,kevinnorwood/2543689,"Norwood, Kevin",WR,2015,Regular Season,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
19069,torreysmith/2495459,"Smith, Torrey",WR,2011,Regular Season,1,1,6,77,1
19070,torreysmith/2495459,"Smith, Torrey",WR,2011,Regular Season,1,1,2,38,0
19071,torreysmith/2495459,"Smith, Torrey",WR,2011,Regular Season,1,1,5,33,0
19072,torreysmith/2495459,"Smith, Torrey",WR,2011,Postseason,1,1,1,9,0


In [19]:
#DATA FRAME THAT contains unique players RB and FB

recivers_id = receiver_clean_df['Player_id'].unique()
recivers_name = receiver_clean_df['Name'].unique()

#qb_id_name = pd.DataFrame(qb_id,qb_name, columns  = ['Player_id','Player_id'])
runners_id_name = pd.DataFrame({'Player_id': qb_id, 'Name' : qb_name})#, columns  = ['Player_id','Player_id'])
runners_id_name


,Player_id,Name
0,billdemory/2512778,"Demory, Bill"
1,tomsavage/2543640,"Savage, Tom"
2,tommymaddox/2501842,"Maddox, Tommy"
3,jimbobtaylor/2527093,"Taylor, Jim Bob"
4,jaybarker/2499535,"Barker, Jay"
...,...,...
462,alpastrana/2522827,"Pastrana, Al"
463,aaronrodgers/2506363,"Rodgers, Aaron"
464,chrisweinke/2504686,"Weinke, Chris"
465,brockosweiler/2533436,"Osweiler, Brock"


In [20]:
conn_string = 'postgres:postgres@localhost:5432/NFL_Statistics'
engine = create_engine(f'postgresql://{conn_string}')
Base = declarative_base()

class RB(Base):
    __tablename__ = 'RB'
    Player_id = Column(String, primary_key = True)
    name = Column(String)
    Position = Column(String)
    Year = Column(Integer)
    Season = Column(String)
    Games_Played = Column(Integer)
    Games_Started = Column(Integer)
    Rushing_Yards = Column(Integer)
    Rushing_Attempts = Column(Integer)

In [21]:
Base.metadata.tables

immutabledict({})

In [22]:
# This is where we create our tables in the database
Base.metadata.create_all(engine)

In [23]:
from sqlalchemy.orm import Session
session = Session(engine)

In [26]:
#pd.read_sql('SELECT * FROM "QB_NAMES"', engine)


In [27]:
#session.new
qb_df_sql.to_sql(name = 'QB',con=engine,if_exists='replace', index=False)

In [28]:
#QB_NAMES
qb_id_name.to_sql(name="QB_NAMES",con=engine,if_exists='replace', index=False)

In [29]:
qb_id_name

,Player_id,Name,Position
0,billdemory/2512778,"Demory, Bill",QB
1,tomsavage/2543640,"Savage, Tom",QB
2,tommymaddox/2501842,"Maddox, Tommy",QB
3,jimbobtaylor/2527093,"Taylor, Jim Bob",QB
4,jaybarker/2499535,"Barker, Jay",QB
...,...,...,...
462,alpastrana/2522827,"Pastrana, Al",QB
463,aaronrodgers/2506363,"Rodgers, Aaron",QB
464,chrisweinke/2504686,"Weinke, Chris",QB
465,brockosweiler/2533436,"Osweiler, Brock",QB


In [30]:

session.close()